In [3]:
# % load_ext autoreload
# % aimport utils
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, timedelta
from utils import *
from sklearn.decomposition import PCA
import plotly.express as px

In [4]:
% autoreload 1
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv',
           error_bad_lines=False)
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
                   error_bad_lines=False)
#cases['FIPS'] = cases['FIPS'].astype('int')
day_cases = get_day_cases()
counties = pd.read_csv('https://raw.githubusercontent.com/Yu-Group/covid19-severity-prediction/master/data/county_data_abridged.csv',
                      error_bad_lines=False)
county_cases = pd.read_csv('https://raw.githubusercontent.com/Yu-Group/covid19-severity-prediction/master/data/county_level/processed/nytimes_infections/nytimes_infections.csv',
                          error_bad_lines=False)

UsageError: Line magic function `%` not found.


In [ ]:
death_keep = [col+'_deaths' if (col in cases.columns and col != 'FIPS') else col for col in deaths.columns]
cases_keep = [col+'_cases' for col in cases.columns[12:]]
cases_and_deaths = cases.merge(deaths, on='FIPS', suffixes=('_cases', '_deaths'))[death_keep+cases_keep]
renamer = lambda name: name if (type(name) != str or '/' in name or '_deaths' not in name) else name[:-7]
cases_and_deaths = cases_and_deaths.rename(columns=renamer)
cases_and_deaths.head()

In [ ]:
valid_territories = cases_and_deaths[(cases_and_deaths['Lat'] > 0) & ~np.isnan(cases_and_deaths['FIPS'])]
valid_territories.loc[:, 'FIPS'] = valid_territories['FIPS'].astype('int')
valid_counties = counties[~counties['countyFIPS'].isin(['City1', 'City2'])]
valid_counties.loc[:, 'countyFIPS'] = valid_counties['countyFIPS'].astype('int')

In [ ]:
# MOST IMPORTANT TABLE
county_all = valid_counties.merge(valid_territories, left_on='countyFIPS', right_on='FIPS')
county_all = county_all.drop(columns=['Lat', 'Long_', 'Province_State', 'Country_Region', 'FIPS'])
#county_all.to_csv('county_info_cases_deaths.csv')

In [ ]:
#sns.jointplot(county_all['lon'],
#              np.sqrt((county_all['4/20/20_cases'] / county_all['Population'])))
d = county_all[county_all['lat'] > 0]
sns.jointplot(np.sqrt(d['lat']),
                np.sqrt(d['5/8/20_deaths'] / d['Population']),
              )

plt.ylim(ymin=0)
plt.show()

In [5]:
def preprocess_counties(counties_df):
    # turn ordinal dates into useful dates
    counties_df = counties_df.iloc[:, 75:83].dropna().apply(lambda col: col.apply(lambda x: pd.Timestamp.fromordinal(int(x))))
    return counties_df

In [6]:
def moving_average(a, n=7, ndim=1) :
    """
    From https://stackoverflow.com/questions/14313510/how-to-calculate-moving-average-using-numpy
    """
    if ndim == 2:
        return moving_average_2d(a, n=n)
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def moving_average_2d(a, n=7):
    ret = np.cumsum(a, axis=1, dtype=float)
    ret[:, n:] = ret[:, n:] - ret[:, :-n]
    return ret[:, n - 1:] / n

def plot_feature_bins(df, colname, ylabel=None, num_bins=4,
                      stat='cases', smoothing=5, normalize=False,
                     quartiles=True, transformer=lambda x: x):
    indep_var = df[colname].astype('float')
    pops = df['PopTotalMale2017'] + df['PopTotalFemale2017']
    if normalize:
        indep_var /= pops
    valid_idxs = ~np.isnan(indep_var)
    indep_var = transformer(indep_var[valid_idxs])
    valid_df = df[valid_idxs]
    
    ivar_min, ivar_max = np.min(indep_var), np.max(indep_var)
    if quartiles:
        bins = [np.percentile(indep_var, 100/num_bins*i) for i in range(num_bins+1)]
    else:
        bins = np.linspace(ivar_min, ivar_max, num_bins+1)
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111)
    for i in range(num_bins):
        binned_idxs = (bins[i] <= indep_var) & (indep_var < bins[i+1])
        binned = valid_df.loc[binned_idxs].filter(regex='_'+stat)
        pop = (valid_df['PopTotalMale2017'] + valid_df['PopTotalFemale2017'])[binned_idxs]
        if stat=='cases':
            #print(binned)
            dates = pd.to_datetime(binned.columns.str.replace(r'_cases', ''))
            binned = (np.array(binned) / pop[:, None]).mean(axis=0)
            binned = pd.Series(data=binned, index=dates)
        else:
            dates = pd.to_datetime(binned.columns.str.replace(r'_deaths', ''))
            binned = (np.array(binned) / pop[:, None]).mean(axis=0)
            binned = pd.Series(data=binned, index=dates)
        binned = binned.astype('float')
        y = np.diff(binned.values)
        x = binned.index[:-smoothing]
        y = moving_average(y, smoothing)
        sns.lineplot(x=x, y=y, ax=ax, label=str(100/num_bins*i) + '-' + str(100/num_bins*(i+1)))
    plt.xlabel('Time')
    if ylabel is None:
        ylabel = 'Cases' if stat=='cases' else 'Deaths'
    plt.setp(ax.get_xticklabels(), rotation=60)
    ax.legend()
    plt.legend(title = colname)
    plt.show()
    return bins
    
    
def find_peak(df, smoothness=2, stat='cases'):
    df = df.filter(regex='_' + stat)
    case_change_arr = np.diff(np.array(df), axis=1)
    smoothed_ccarr = moving_average(case_change_arr, n=smoothness, ndim=2)
    print(smoothed_ccarr.shape)
    peak_dates = df.columns[np.argmax(smoothed_ccarr, axis=1)+1].str.replace('_' + stat, '')
    return pd.to_datetime(peak_dates)

In [7]:
# cell to play around with!
#plot_feature_bins(has_both, 'PopTotalMale2017', stat='cases', smoothing=7, normalize=True)
bins1 = plot_feature_bins(has_both, 'dem_to_rep_ratio', stat='deaths',
                          smoothing=7, normalize=False)
bins2 = plot_feature_bins(has_both, 'dem_to_rep_ratio', stat='cases',
                          smoothing=7, normalize=False)
print('Bins1:', bins1)
print('Bins2:', bins2)

NameError: name 'has_both' is not defined

In [8]:
has_both[has_both['dem_to_rep_ratio'] > 1]

NameError: name 'has_both' is not defined

In [9]:
has_both.iloc[:5, 93:207]

NameError: name 'has_both' is not defined

In [ ]:
n = 7
has_both = county_all[(np.sum(county_all.iloc[:, 95:205], axis=1) > 0) & (np.sum(county_all.iloc[:, 205:], axis=1) > 0)]
has_cases = county_all[(np.sum(county_all.iloc[:, 95:205], axis=1) > 0) | (np.sum(county_all.iloc[:, 205:], axis=1) > 0)]
deltas = (find_peak(has_both, stat='deaths', smoothness=n) -
          find_peak(has_both, stat='cases', smoothness=n)).days
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111)
ax.axvline(0, color='red', linestyle='--')
sns.distplot(np.array(deltas), kde=True, ax=ax)
print('Median Case/Death Lag:', np.median(deltas))
print('Mean Case/Death Lag:', np.mean(deltas))
plt.show()

In [ ]:
def corr(df, var1, var2):
    demean = lambda arr: arr - arr.mean()
    col1, col2 = demean(has_cases[var1]), demean(has_cases[var2])
    return np.sum(col1*col2)/np.sqrt(np.sum(col1**2) * np.sum(col2**2))

for col in has_cases.columns[:94]:
    if has_cases[col].dtype != object and np.count_nonzero(np.isnan(has_cases[col])):    
        var1 = '5/11/20_cases'
        var2 = '5/11/20_deaths'
        print('Column:\t\t\t\t', col)
        print('Correlation with Cases:\t\t', corr(has_cases, col, var1))
        print('Correlation with Deaths:\t', corr(has_cases, col, var2))
        print()

In [ ]:
[col for col in county_all.columns if 'pop' in col.lower()]

In [ ]:
def preprocess(arr, smoothness=7):
    """
    Differentiates and smooths the array, and excludes whatever indices are meant to be excluded.
    """
    return moving_average(np.diff(arr[keep_idxs(arr, 1849)], axis=1), n=smoothness, ndim=2)

county_all_deaths = np.array(county_all.iloc[:, 95:205]) / county_all['PopulationEstimate2018'][:, None]
county_all_deaths = preprocess(county_all_deaths)
county_deaths_centered = county_all_deaths - county_all_deaths.mean(axis=1)[:, None]
county_all_cases = np.array(county_all.iloc[:, 205:]) / county_all['PopulationEstimate2018'][:, None]
county_all_cases = preprocess(county_all_cases)
county_cases_centered = county_all_cases - county_all_cases.mean(axis=1)[:, None]
    
keep_idxs = lambda arr, excl: (np.arange(len(arr)) != excl)

names_nony = (county_all['CountyName'] + ', ' + county_all['StateName']).iloc[keep_idxs(county_all, 1849)]
county_all_nony = county_all[keep_idxs(county_all, 1849)]

In [ ]:
print(county_deaths_nony.shape)

### Note of data cleaning

We only consider assigned counties, as their time series include far fewer negative jumps, which are seen to be inaccurate.

For PCA we also exclude New York (it gets its own "cluster").

In [ ]:
deaths.columns

In [ ]:
death_nums = np.array(county_all.iloc[:, 95:206])
day_diffs = np.diff(death_nums, axis=1)
negs = np.argwhere(day_diffs < 0)
case_nums = np.array(county_all.iloc[:, 206:])
day_diffs_cases = np.diff(case_nums, axis=1)
negs_cases = np.argwhere(day_diffs_cases < 0)

In [ ]:
negs_cases.shape

In [ ]:
for pair in negs_cases:
    val = day_diffs_cases[pair[0], pair[1]]
    # Only look at changes that aren't that 
    if val < -10:
        print('County Name:', county_all['CountyName'][pair[0]])
        print('Decrease in Number:', val)
        print()

In [ ]:
k = 1863
plt.plot(np.arange(len(deaths.iloc[k, 15:])), deaths.iloc[k, 15:])

In [ ]:
pca = PCA(n_components=2, whiten=False)
pca.fit(county_all_deaths)

In [ ]:
p = pca.fit_transform(county_deaths_centered)
px.scatter(p[:, 0], p[:, 1], hover_name=list(names_nony), color=county_all_nony['lat'])

In [ ]:
px.scatter(p[:, 0], p[:, 1], hover_name=list(names_nony), color=county_all_nony['lon'])

In [ ]:
for col in county_all_cases.columns:
    if county_all_cases[col].dtype != 'int64':
        print(county_all_cases[col].dtype)

In [ ]:
p = pca(county_all_cases, 2)

In [ ]:
county_all_cases.shape